### Machine Learning

## ! update code

In [1]:
# wip, refactor as we firm model choices
# Import libraries for pre-processing
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

### Import files

In [2]:
# Import the files
df_test = pd.read_csv("https://raw.githubusercontent.com/AngShengJun/dsi14P4/master/assets/working/df_test_weather_cleaned.csv")
df_train = pd.read_csv("https://raw.githubusercontent.com/AngShengJun/dsi14P4/master/assets/working/df_train_weather_cleaned.csv")

### Prep

In [3]:
# Make a copy of test
df_testcopy = df_test.copy()

In [4]:
# Review the 1st line
df_testcopy.head(1)

,Unnamed: 0,id,date,lat,long,tavg,stnpress,dewpt,precip,windspeed,...,trap_T230,trap_T231,trap_T232,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903
0,0,1,2008-06-11,41.95469,-87.800991,75.0,29.31,55.5,0.0,9.15,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Drop unnecessary col
df_testcopy.drop(['id','Unnamed: 0'],axis=1,inplace=True)
# Review
df_testcopy.head(1)

,date,lat,long,tavg,stnpress,dewpt,precip,windspeed,daylight,spc_CULEX PIPIENS,...,trap_T230,trap_T231,trap_T232,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903
0,2008-06-11,41.95469,-87.800991,75.0,29.31,55.5,0.0,9.15,15.166667,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Make a copy of train
df_traincopy = df_train.copy()

In [ ]:
# Review the 1st line
df_traincopy.head(1)

In [ ]:
# Drop unnecessary col
df_traincopy.drop(['Unnamed: 0'],axis=1,inplace=True)
# Review
df_traincopy.head(1)

In [ ]:
df_traincopy.index


In [ ]:
# Sanity check
print(df_traincopy.shape)
print(df_testcopy.shape)

In [ ]:
# Drop data column from test and train
df_traincopy.drop(['date'],axis=1,inplace=True)
df_testcopy.drop(['date'],axis=1,inplace=True)

In [ ]:
X_trc = df_traincopy.drop(['num_mos','wnv'],axis=1)
y_trc = df_traincopy['wnv']

### Train-Validate-Split

In [ ]:
# Train-validate-split
X_train,X_val,y_train,y_val = train_test_split(X_trc,y_trc,test_size=0.3,random_state=42, stratify=y_trc)

In [ ]:
print(y_train.value_counts(normalize=True))
y_val.value_counts(normalize=True)

### Resampling

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_sample(X_train, y_train)

In [ ]:
print(y_sm.value_counts(normalize=True))
y_val.value_counts(normalize=True)

### Classification

### Logistic Regression Model

In [ ]:
pipe1 = Pipeline([
    ('ss', StandardScaler()),
    ('lor', LogisticRegression(solver='lbfgs',random_state=42)),
])

In [ ]:
pipe1.get_params()

In [ ]:
# Define the pipe parameters
pipe1_params = {'ss__with_mean': [True],
               'ss__with_std': [True],
               'lor__max_iter': [500]}

In [ ]:
# Instantiate Gridsearch
gs1 = GridSearchCV(pipe1,\
                  param_grid=pipe_params,\
                  cv=10)
# Fit GridSearch to the cleaned training data.
gs1.fit(X_sm,y_sm)

In [ ]:
# Check the results of the grid search

print(f"Best parameters: {gs1.best_params_}")
print(f"Best score: {gs1.best_score_}")

In [ ]:
model1 = gs1.best_estimator_

In [ ]:
# Score model on train set and validate set
print(f"Accuracy on train set: {model1.score(X_sm, y_sm)}")
print(f"Accuracy on validate set: {model1.score(X_val, y_val)}")

### Decision Tree

In [ ]:
pipe4 = Pipeline([
    ('ss', StandardScaler()),
    ('dtc', DecisionTreeClassifier())
])

In [ ]:
pipe4.get_params()

### KNN Classifier

In [ ]:
#Skip MultinomialNB when some of the features have negative values.
#Use GaussianNB instead.
pipe3 = Pipeline([
    ('ss', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=5))
])

In [ ]:
pipe3.get_params()

In [ ]:
# Define the pipe parameters
pipe3_params = {'ss__with_mean': [True],
               'ss__with_std': [True],
               }

In [ ]:
# Instantiate Gridsearch
gs3 = GridSearchCV(pipe3,\
                  param_grid=pipe3_params,\
                  cv=10)
# Fit GridSearch to the cleaned training data.
gs3.fit(X_sm,y_sm)

In [ ]:
# Define X and y (for prediction)
X_tr = df_traincopy.drop(['num_mos','wnv'],axis=1)
y_tr = df_traincopy['num_mos']

In [ ]:
print(y_trc.value_counts(normalize=True))
y_val.value_counts(normalize=True)

### Train-Validate-Split

In [ ]:

# Train-validate-split
X_train,X_val,y_train,y_val = train_test_split(X_tr,y_tr,test_size=0.3,random_state=42)

In [ ]:
# Initiate standard scaler
ss = StandardScaler()
# fit on X_train
ss.fit(X_train)
# transform X_train
X_trainsc = ss.transform(X_train)
# transform X_val
X_valsc = ss.transform(X_val)

In [ ]:
# Import regression models
from sklearn.linear_model import Ridge, Lasso, LinearRegression, RidgeCV, LassoCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
# Libraries for model evaluate
from sklearn.metrics import mean_squared_error

### Use pipeline + Gridsearch

In [ ]:
# Instantiate models use default settings

lr = LinearRegression()
lasso = LassoCV()
ridge = RidgeCV()
knn = KNeighborsRegressor()
cart = DecisionTreeRegressor()
bagged = BaggingRegressor()
random_forest_reg = RandomForestRegressor()
adaboost_reg = AdaBoostRegressor()
support_vector_reg = SVR()


In [ ]:
cart = DecisionTreeRegressor()

In [ ]:
cart_scores = cross_val_score(cart, X_trainsc, y_train, cv=10)
cart_scores.mean()

In [ ]:
lr_scores = cross_val_score(lr, X_trainsc, y_train, cv=10)
lr_scores.mean()

In [ ]:
lasso_scores = cross_val_score(lasso,  X_trainsc, y_train, cv=10)
lasso_scores.mean()

In [ ]:
ridge_scores = cross_val_score(ridge, X_train, y_train, cv=3)
ridge_scores.mean()

*italicized text*### Classify Wnv class

In [ ]:
# Define X and y (for classification)

## Baseline Accuracy 

In [ ]:
y_test.value_counts(normalize = True)

Discuss what metric we want to optimize. F1, ROC_AUC, Sensitivity, Precision, Mis-class rate, or Specificity. 
Type II error worse right? In interest of public health, we want to minimise False Negative rate.